In [1]:
import os

MODEL_DIRECTORY_1 = "/kaggle/input/simclr-model/runs/May26_22-27-46_ca41811d77bd/"
MODEL_DIRECTORY_2 = "/kaggle/input/simclr-model-100/runs/May27_10-38-58_a9089d95001b/"

checkpoint_files = []
for ckpt_path in (MODEL_DIRECTORY_1, MODEL_DIRECTORY_2):
    checkpoint_files += [(f,ckpt_path) for f in os.listdir(ckpt_path) if f.endswith('.pth.tar')]

checkpoint_paths_list = [os.path.join(ckpt_path, f) for f,ckpt_path in checkpoint_files]
checkpoint_paths_list = sorted(checkpoint_paths_list, key=lambda x: os.path.basename(x))

for path in checkpoint_paths_list:
    print(path)


/kaggle/input/simclr-model/runs/May26_22-27-46_ca41811d77bd/checkpoint_epoch_0005.pth.tar
/kaggle/input/simclr-model/runs/May26_22-27-46_ca41811d77bd/checkpoint_epoch_0010.pth.tar
/kaggle/input/simclr-model/runs/May26_22-27-46_ca41811d77bd/checkpoint_epoch_0015.pth.tar
/kaggle/input/simclr-model/runs/May26_22-27-46_ca41811d77bd/checkpoint_epoch_0020.pth.tar
/kaggle/input/simclr-model/runs/May26_22-27-46_ca41811d77bd/checkpoint_epoch_0025.pth.tar
/kaggle/input/simclr-model/runs/May26_22-27-46_ca41811d77bd/checkpoint_epoch_0030.pth.tar
/kaggle/input/simclr-model/runs/May26_22-27-46_ca41811d77bd/checkpoint_epoch_0035.pth.tar
/kaggle/input/simclr-model/runs/May26_22-27-46_ca41811d77bd/checkpoint_epoch_0040.pth.tar
/kaggle/input/simclr-model/runs/May26_22-27-46_ca41811d77bd/checkpoint_epoch_0045.pth.tar
/kaggle/input/simclr-model/runs/May26_22-27-46_ca41811d77bd/checkpoint_epoch_0050.pth.tar
/kaggle/input/simclr-model-100/runs/May27_10-38-58_a9089d95001b/checkpoint_epoch_0055.pth.tar
/kaggl

In [2]:
# Code for clearing memory

import torch
import gc

gc.collect()
torch.cuda.empty_cache()
if torch.cuda.is_available():
    torch.cuda.ipc_collect()

In [3]:

import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.amp import GradScaler, autocast
from torch.utils.tensorboard import SummaryWriter
from torchvision.transforms import transforms
import torchvision.models as models
import torch.backends.cudnn as cudnn
from torchvision import datasets

from tqdm import tqdm
# 
import os
import sys
import logging

import yaml
import argparse

torch.manual_seed(0)
np.random.seed(0)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

2025-05-29 05:29:27.047608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748496567.242200      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748496567.297580      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


device(type='cuda')

In [4]:
import os
from torchvision import datasets, transforms
from torch.utils.data import ConcatDataset, DataLoader
from tqdm import tqdm

base_path = "/kaggle/input/ssl-dataset/ssl_dataset"
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

all_class_names = set()
for i in range(1, 5):
    split_path = os.path.join(base_path, f"train.X{i}")
    all_class_names.update(os.listdir(split_path))

all_class_names = sorted(all_class_names)
class_to_idx = {cls_name: idx for idx, cls_name in enumerate(all_class_names)}


# train_datasets = []
# for i in tqdm(range(1, 5)):
#     folder = f"train.X{i}"
#     split_path = os.path.join(base_path, folder)
#     dataset = datasets.ImageFolder(split_path, transform=transform)
#     dataset.class_to_idx = class_to_idx
#     dataset.samples = [(path, class_to_idx[os.path.basename(os.path.dirname(path))]) 
#                        for path, _ in dataset.samples]
#     train_datasets.append(dataset)

# full_train_dataset = ConcatDataset(train_datasets)

val_dataset = datasets.ImageFolder("/kaggle/input/ssl-dataset/ssl_dataset/val.X", transform=transform)
val_dataset.class_to_idx = class_to_idx

In [5]:
import json

output_path = '/kaggle/working/class_to_idx.json'

with open(output_path, 'w') as f:
    json.dump(class_to_idx, f)

print(f"Saved class_to_idx to {output_path}")

Saved class_to_idx to /kaggle/working/class_to_idx.json


In [6]:
# exceptions/exceptions.py

class BaseSimCLRException(Exception):
    """Base exception"""


class InvalidBackboneError(BaseSimCLRException):
    """Raised when the choice of backbone Convnet is invalid."""


class InvalidDatasetSelection(BaseSimCLRException):
    """Raised when the choice of dataset is invalid."""

In [7]:
class ResNetSimCLR(nn.Module):
    def __init__(self, base_model, out_dim):
        super(ResNetSimCLR, self).__init__()
        self.resnet_dict = {"resnet18": models.resnet18(pretrained=False, num_classes=out_dim),
                            "resnet50": models.resnet50(pretrained=False, num_classes=out_dim)}

        self.backbone = self._get_basemodel(base_model)
        dim_mlp = self.backbone.fc.in_features

        self.backbone.fc = nn.Sequential(nn.Linear(dim_mlp, dim_mlp), nn.ReLU(), self.backbone.fc)

    def _get_basemodel(self, model_name = 'resnet50'):
        try:
            model = self.resnet_dict[model_name]
        except KeyError:
            raise InvalidBackboneError(
                "Invalid backbone architecture. Check the config file and pass one of: resnet18 or resnet50")
        else:
            return model

    def forward(self, x):
        return self.backbone(x)

In [8]:
# Loading model and converting it to feature extractor

import torch
import torch.nn as nn
import numpy as np
from torchvision import models
from tqdm import tqdm

for checkpoint_path in checkpoint_paths_list:
    file_name = os.path.basename(checkpoint_path)
    checkpoint_num = os.path.splitext(os.path.splitext(file_name)[0])[0][-3:]

    # train_loader = DataLoader(full_train_dataset, batch_size=256, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=256, shuffle=True, num_workers=4)
    model = ResNetSimCLR(base_model="resnet50", out_dim=128)
    checkpoint = torch.load(checkpoint_path)
    
    if torch.cuda.device_count() > 1:
            print(f"Using {torch.cuda.device_count()} GPUs.")
            model = torch.nn.DataParallel(model) # Since model was stored as data parallel
    
    model.load_state_dict(checkpoint['state_dict'])
    
    num_classes = len(all_class_names)
    feature_dim = model.module.backbone.fc[0].in_features
    encoder = model.module.backbone
    encoder.fc = nn.Identity()
    encoder.to(DEVICE)
    
    if torch.cuda.device_count() > 1:
            print(f"Using {torch.cuda.device_count()} GPUs.")
            encoder = torch.nn.DataParallel(encoder)
    
    all_features = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader):
            inputs = inputs.to(DEVICE)
            features = encoder(inputs)
            all_features.append(features.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    
    features_array = np.concatenate(all_features, axis=0)
    labels_array = np.concatenate(all_labels, axis=0)

    os.makedirs('/kaggle/working/extracted_features', exist_ok=True)
    np.savez_compressed(f'/kaggle/working/extracted_features/val_data_checkpoint_{checkpoint_num}.npz', features=features_array, labels=labels_array)
    
    print(f"Saved features and labels together in /kaggle/working/val_data_checkpoint_{checkpoint_num}.npz")

    del model
    del encoder
    torch.cuda.empty_cache()
    gc.collect()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Using 2 GPUs.
Using 2 GPUs.


100%|██████████| 20/20 [00:19<00:00,  1.02it/s]


Saved features and labels together in /kaggle/working/val_data_checkpoint_005.npz
Using 2 GPUs.
Using 2 GPUs.


100%|██████████| 20/20 [00:17<00:00,  1.15it/s]


...

Saved features and labels together in /kaggle/working/val_data_checkpoint_090.npz
Using 2 GPUs.
Using 2 GPUs.


100%|██████████| 20/20 [00:17<00:00,  1.16it/s]


Saved features and labels together in /kaggle/working/val_data_checkpoint_095.npz
Using 2 GPUs.
Using 2 GPUs.


100%|██████████| 20/20 [00:17<00:00,  1.14it/s]


Saved features and labels together in /kaggle/working/val_data_checkpoint_100.npz
